<a href="https://colab.research.google.com/github/KarinaPl10/Projects/blob/main/streamlit_app_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
"""
Streamlit - Gestor Inteligente de Proyecto (Prototipo) - REVISADO

Este archivo mantiene dos modos de ejecución:
1) Modo Streamlit (interfaz web) — si tienes streamlit instalado y ejecutas con `streamlit run`.
2) Modo no interactivo / demostración — si NO tienes un terminal interactivo disponible
   (p. ej. entornos sandbox donde `input()` falla). En este modo se ejecuta una demostración automatizada
   que prueba la lógica del gestor sin pedir entradas al usuario.

Objetivos de esta revisión:
- Evitar usar input() en entornos no interactivos (previene OSError: [Errno 29] I/O error).
- Ofrecer un fallback no interactivo que ejecute una demo automatizada y muestre resultados.
- Mantener la UI de Streamlit si `streamlit` está disponible.

Uso:
- Interfaz web (recomendada para interacción):
    pip install streamlit
    streamlit run streamlit_project_manager.py

- Demo no interactiva (por ejemplo en entornos sandbox):
    python streamlit_project_manager.py

El script detecta automáticamente si el entorno es interactivo (terminal) y si streamlit está instalado.

Si quieres un comportamiento interactivo pero tu entorno no permite `input()`, dime en el chat cómo
quieres que se recepcionen las entradas (por ejemplo: mediante archivos con comandos, variables de entorno,
endpoints HTTP, o una interfaz web con FastAPI + React) y lo implementaré.
"""

import json
import base64
import os
import sys
import tempfile
from typing import Dict, List, Optional

# Intentar importar streamlit; si no está disponible, usaremos modo demo/CLI fallback
try:
    import streamlit as st
    STREAMLIT_AVAILABLE = True
except ModuleNotFoundError:
    STREAMLIT_AVAILABLE = False

# Detectar si stdin es interactivo (terminal). Si no, evitamos cualquier input() call.
STDIN_IS_INTERACTIVE = sys.stdin.isatty()

# ------------------ LÓGICA DEL AGENTE / MODELO DE DOMINIO ------------------
class ProjectPhaseAgentLocal:
    def __init__(self):
        self.tasks: Dict[str, List[str]] = {
            "Visión": [
                "Definir objetivos del proyecto",
                "Identificar stakeholders",
                "Establecer criterios de éxito"
            ],
            "Planeación": [
                "Diseñar cronograma",
                "Asignar recursos",
                "Definir responsables"
            ],
            "Ejecución": [
                "Implementar plan de acción",
                "Monitorear progreso",
                "Ajustar desviaciones"
            ]
        }

    def add_phase(self, phase_name: str, tasks: Optional[List[str]] = None):
        if tasks is None:
            tasks = []
        self.tasks[phase_name] = tasks

    def get_phases(self) -> List[str]:
        return list(self.tasks.keys())

    def get_tasks(self, phase: str) -> List[str]:
        return self.tasks.get(phase, [])


# ------------------ ESTADO Y SERIALIZACIÓN INDEPENDIENTE DE UI ------------------
class ProjectState:
    def __init__(self, agent: ProjectPhaseAgentLocal):
        self.agent = agent
        self.current_phase: str = "Visión"
        self.completed: Dict[str, set] = {phase: set() for phase in self.agent.get_phases()}
        self.uploaded_files: Dict[str, Dict[int, List[Dict[str, bytes]]]] = {phase: {} for phase in self.agent.get_phases()}
        self.log: List[str] = []

    def ensure_phase_initialized(self, phase: str):
        self.completed.setdefault(phase, set())
        self.uploaded_files.setdefault(phase, {})

    def toggle_task(self, phase: str, index: int, checked: bool):
        self.ensure_phase_initialized(phase)
        if checked:
            self.completed[phase].add(index)
            self.log.append(f"Tarea {index} en '{phase}' marcada como COMPLETADA")
        else:
            self.completed[phase].discard(index)
            self.log.append(f"Tarea {index} en '{phase}' desmarcada")

    def next_task(self, phase: str):
        tasks = self.agent.get_tasks(phase)
        completed = self.completed.get(phase, set())
        for i, t in enumerate(tasks):
            if i not in completed:
                self.log.append(f"Siguiente tarea en '{phase}' -> {t}")
                return i, t
        self.log.append(f"No quedan tareas pendientes en '{phase}'")
        return None, None

    def upload_file_for_task(self, phase: str, index: int, name: str, data: bytes):
        self.ensure_phase_initialized(phase)
        dest = self.uploaded_files.setdefault(phase, {})
        dest.setdefault(index, [])
        dest[index].append({"name": name, "data": data})
        self.log.append(f"Archivo '{name}' subido para tarea {index} en '{phase}'")

    def to_serializable(self) -> Dict:
        return {
            "current_phase": self.current_phase,
            "completed": {p: list(idx) for p, idx in self.completed.items()},
            "uploaded_files": {
                p: {
                    str(i): [{"name": f["name"], "data_b64": base64.b64encode(f["data"]).decode()} for f in files]
                    for i, files in mapping.items()
                }
                for p, mapping in self.uploaded_files.items()
            }
        }

    def save_to_file(self, filename: str = "project_state.json"):
        try:
            serializable = self.to_serializable()
            with open(filename, "w", encoding="utf-8") as f:
                json.dump(serializable, f, ensure_ascii=False, indent=2)
            self.log.append(f"Estado guardado en {filename}")
        except Exception as e:
            self.log.append(f"Error al guardar estado en {filename}: {e}")

    def load_from_file(self, filename: str = "project_state.json"):
        try:
            with open(filename, "r", encoding="utf-8") as f:
                data = json.load(f)
            self.current_phase = data.get("current_phase", self.current_phase)
            self.completed = {p: set(v) for p, v in data.get("completed", {}).items()}

            uf = {}
            for p, mapping in data.get("uploaded_files", {}).items():
                uf[p] = {}
                for i_str, files in mapping.items():
                    i = int(i_str)
                    uf[p][i] = []
                    for f in files:
                        uf[p][i].append({"name": f["name"], "data": base64.b64decode(f["data_b64"])})
            self.uploaded_files = uf
            self.log.append(f"Estado cargado desde {filename}")
        except FileNotFoundError:
            self.log.append(f"Archivo {filename} no encontrado")
        except Exception as e:
            self.log.append(f"Error al cargar estado desde {filename}: {e}")


# ------------------ STREAMLIT UI ------------------
def run_streamlit_app(state: ProjectState):
    st.set_page_config(page_title="Gestor Inteligente de Proyecto", layout="wide")

    if "ps_state" not in st.session_state:
        st.session_state.ps_state = state

    ps: ProjectState = st.session_state.ps_state
    agent = ps.agent

    # Sidebar
    with st.sidebar:
        st.header("Controles del proyecto")

        phases = agent.get_phases()
        index = phases.index(ps.current_phase) if ps.current_phase in phases else 0
        selected_phase = st.selectbox("Selecciona fase", phases, index=index)
        if selected_phase != ps.current_phase:
            ps.current_phase = selected_phase
            ps.log.append(f"Fase seleccionada: {selected_phase}")

        st.markdown("---")

        with st.expander("Añadir nueva fase"):
            new_phase = st.text_input("Nombre nueva fase", key="new_phase_name")
            new_tasks_raw = st.text_area("Lista de tareas (una por línea)")
            if st.button("Crear fase"):
                tasks_list = [t.strip() for t in new_tasks_raw.splitlines() if t.strip()]
                if new_phase:
                    agent.add_phase(new_phase, tasks=tasks_list)
                    ps.ensure_phase_initialized(new_phase)
                    st.success(f"Fase '{new_phase}' creada con {len(tasks_list)} tareas")
                    st.experimental_rerun()
                else:
                    st.error("Escribe el nombre de la fase")

        st.markdown("---")
        if st.button("Siguiente tarea (automática)"):
            i, t = ps.next_task(ps.current_phase)
            if t is not None:
                st.info(f"Siguiente tarea: {t}")
            else:
                st.info("No quedan tareas pendientes en esta fase")

        if st.button("Guardar estado (project_state.json)"):
            ps.save_to_file()
            st.success("Guardado")

        if st.button("Cargar estado (project_state.json)"):
            ps.load_from_file()
            st.experimental_rerun()

        st.markdown("---")
        st.header("Registro (últimas acciones)")
        for entry in ps.log[-10:][::-1]:
            st.write(entry)

    # Main page
    st.title("🧭 Gestor Inteligente de Proyecto - Prototipo")
    st.caption("Marca tareas, sube documentos por tarea, exporta/importa el estado del proyecto.")

    current_phase = ps.current_phase
    st.subheader(f"Fase actual: {current_phase}")

    tasks = agent.get_tasks(current_phase)
    completed_count = len(ps.completed.get(current_phase, set()))
    total = len(tasks)
    progress_pct = int((completed_count / total) * 100) if total > 0 else 0
    st.progress(progress_pct / 100)
    st.write(f"Progreso: {completed_count} / {total}  ({progress_pct}%)")

    st.markdown("---")

    for i, t in enumerate(tasks):
        cols = st.columns([0.05, 0.75, 0.2])
        checked = i in ps.completed.get(current_phase, set())
        new_checked = cols[0].checkbox("", value=checked, key=f"chk_{current_phase}_{i}")
        if new_checked != checked:
            ps.toggle_task(current_phase, i, new_checked)
        cols[1].markdown(f"**Tarea {i+1}:** {t}")

        upload_key = f"u_{current_phase}_{i}"
        uploaded = cols[2].file_uploader("Subir (opcional)", accept_multiple_files=True, key=upload_key)
        if uploaded:
            for uf in uploaded:
                ps.upload_file_for_task(current_phase, i, uf.name, uf.getvalue())

        files_for_task = ps.uploaded_files.get(current_phase, {}).get(i, [])
        if files_for_task:
            with st.expander(f"Archivos ({len(files_for_task)})"):
                for fobj in files_for_task:
                    st.write(fobj["name"])
                    st.download_button(label="Descargar", data=fobj["data"], file_name=fobj["name"])

    st.markdown("---")

    if st.button("Descargar estado como JSON" ):
        json_str = json.dumps(ps.to_serializable(), ensure_ascii=False, indent=2)
        st.download_button("Descargar JSON", data=json_str, file_name="project_state_export.json", mime="application/json")

    st.markdown("---")
    with st.expander("Atajos y acciones rápidas"):
        st.write("- Usa las casillas para marcar tareas como completadas.")
        st.write("- Sube documentos por tarea; estarán disponibles para descargar desde la UI.")
        st.write("- Crea nuevas fases desde la barra lateral.")
        st.write("- Usa 'Siguiente tarea' para que el sistema te indique la próxima tarea pendiente.")

    st.markdown("---")
    with st.expander("Mostrar todo el log"):
        for entry in ps.log[::-1]:
            st.write(entry)

    st.caption("Prototipo - puedes adaptar la lógica para integrarlo a un backend con FastAPI/DB y un frontend en React si deseas escalarlo.")


# ------------------ CLI INTERACTIVO (solo si stdin es TTY) ------------------
def run_interactive_cli_app(state: ProjectState):
    def clear_console():
        os.system('cls' if os.name == 'nt' else 'clear')

    def pause():
        input("\nPresiona Enter para continuar...")

    ps = state
    agent = ps.agent

    while True:
        clear_console()
        print("=== Gestor Inteligente de Proyecto (CLI) ===\n")
        print(f"Fase actual: {ps.current_phase}")
        phases = agent.get_phases()
        print("Fases disponibles:")
        for idx, p in enumerate(phases):
            marker = "<-" if p == ps.current_phase else ""
            print(f"  {idx+1}. {p} {marker}")
        print("\nMenú:")
        print("  1) Seleccionar fase")
        print("  2) Mostrar tareas y progreso")
        print("  3) Mostrar siguiente tarea")
        print("  4) Marcar/Desmarcar tarea")
        print("  5) Subir (registrar) archivo para tarea (ruta/nota)")
        print("  6) Guardar estado (project_state.json)")
        print("  7) Cargar estado (project_state.json)")
        print("  8) Mostrar log")
        print("  9) Añadir nueva fase")
        print("  0) Salir")

        # input() safe here because this function only runs when stdin is a tty
        choice = input("\nElige una opción: ").strip()
        if choice == "1":
            sel = input("Número de fase: ").strip()
            try:
                i = int(sel) - 1
                if 0 <= i < len(phases):
                    ps.current_phase = phases[i]
                    ps.log.append(f"Fase seleccionada: {ps.current_phase}")
                else:
                    print("Índice fuera de rango")
            except ValueError:
                print("Entrada inválida")
            pause()

        elif choice == "2":
            tasks = agent.get_tasks(ps.current_phase)
            completed = ps.completed.get(ps.current_phase, set())
            print(f"\nTareas en fase '{ps.current_phase}':")
            for i, t in enumerate(tasks):
                status = "✅" if i in completed else "⬜"
                print(f"  {i+1}. {status} {t}")
            total = len(tasks)
            completed_count = len(completed)
            pct = int((completed_count / total) * 100) if total > 0 else 0
            print(f"\nProgreso: {completed_count}/{total} ({pct}%)")
            pause()

        elif choice == "3":
            i, t = ps.next_task(ps.current_phase)
            if t is not None:
                print(f"Siguiente tarea: {t} (#{i+1})")
            else:
                print("No quedan tareas pendientes en esta fase")
            pause()

        elif choice == "4":
            tasks = agent.get_tasks(ps.current_phase)
            print("Ingrese número de tarea a (des)marcar:")
            for i, t in enumerate(tasks):
                status = "✅" if i in ps.completed.get(ps.current_phase, set()) else "⬜"
                print(f"  {i+1}. {status} {t}")
            sel = input("Número de tarea: ").strip()
            try:
                idx = int(sel) - 1
                if 0 <= idx < len(tasks):
                    currently = idx in ps.completed.get(ps.current_phase, set())
                    ps.toggle_task(ps.current_phase, idx, not currently)
                    print(f"Tarea {idx+1} actualizada")
                else:
                    print("Índice fuera de rango")
            except ValueError:
                print("Entrada inválida")
            pause()

        elif choice == "5":
            tasks = agent.get_tasks(ps.current_phase)
            print("Ingrese número de tarea para asociar el archivo:")
            for i, t in enumerate(tasks):
                print(f"  {i+1}. {t}")
            sel = input("Número de tarea: ").strip()
            try:
                idx = int(sel) - 1
                if 0 <= idx < len(tasks):
                    path = input("Ruta local del archivo (se guardará el contenido en el estado si existe, o la ruta como nota): ").strip()
                    if os.path.exists(path) and os.path.isfile(path):
                        with open(path, "rb") as f:
                            data = f.read()
                        ps.upload_file_for_task(ps.current_phase, idx, os.path.basename(path), data)
                        print("Archivo leído y guardado en el estado interno")
                    else:
                        ps.upload_file_for_task(ps.current_phase, idx, os.path.basename(path), path.encode())
                        print("Ruta guardada como nota (archivo no leído)")
                else:
                    print("Índice fuera de rango")
            except ValueError:
                print("Entrada inválida")
            pause()

        elif choice == "6":
            ps.save_to_file()
            print("Estado guardado")
            pause()

        elif choice == "7":
            ps.load_from_file()
            print("Intentó cargar el estado (revisa el log para detalles)")
            pause()

        elif choice == "8":
            print("\nLog (últimas entradas):")
            for e in ps.log[-20:]:
                print(f" - {e}")
            pause()

        elif choice == "9":
            name = input("Nombre de la nueva fase: ").strip()
            if not name:
                print("Nombre vacío, cancelado")
                pause()
                continue
            print("Introduce tareas, una por línea. Deja vacío y presiona Enter para terminar.")
            lines = []
            while True:
                line = input()
                if not line.strip():
                    break
                lines.append(line.strip())
            agent.add_phase(name, tasks=lines)
            ps.ensure_phase_initialized(name)
            print(f"Fase '{name}' creada con {len(lines)} tareas")
            pause()

        elif choice == "0":
            print("Saliendo...")
            break
        else:
            print("Opción inválida")
            pause()


# ------------------ DEMO NO INTERACTIVO (para entornos sin TTY) ------------------
def run_demo_noninteractive(state: ProjectState, temp_dir: Optional[str] = None):
    """
    Ejecuta una secuencia automatizada de acciones (sin input) para probar la lógica.
    Esta función se usa en entornos donde input() no está disponible (por ejemplo, sandboxes).
    """
    ps = state
    agent = ps.agent

    print("=== Demo no interactiva del Gestor Inteligente de Proyecto ===")
    print("Fases detectadas:")
    for p in agent.get_phases():
        print(f" - {p}")

    # Mostrar siguiente tarea en Visión
    i, t = ps.next_task("Visión")
    if t:
        print(f"Siguiente tarea en Visión: ({i}) {t}")

    # Marcar primera tarea como completada
    ps.toggle_task("Visión", 0, True)
    print("Marcada como completada: Tarea 1 de Visión")

    # Subir un archivo de prueba (contenido generado) para la tarea 1
    dummy_content = b"Documento de ejemplo para la tarea 1"
    ps.upload_file_for_task("Visión", 0, "ejemplo.txt", dummy_content)
    print("Archivo de ejemplo asociado a la tarea 1 de Visión")

    # Mostrar progreso
    tasks = agent.get_tasks("Visión")
    completed_count = len(ps.completed.get("Visión", set()))
    total = len(tasks)
    pct = int((completed_count / total) * 100) if total > 0 else 0
    print(f"Progreso en Visión: {completed_count}/{total} ({pct}%)")

    # Guardar estado en archivo temporal si es posible
    if temp_dir is None:
        temp_dir = tempfile.gettempdir()
    filename = os.path.join(temp_dir, "project_state_demo.json")
    try:
        ps.save_to_file(filename)
        print(f"Estado guardado en: {filename}")
    except Exception as e:
        print(f"No se pudo guardar el estado en disco: {e}")

    # Mostrar log
    print("\nLog de acciones:")
    for e in ps.log:
        print(f" - {e}")

    print("\nDemo finalizada.")


# ------------------ PUNTO DE ENTRADA ------------------
if __name__ == "__main__":
    agent = ProjectPhaseAgentLocal()
    state = ProjectState(agent)

    if STREAMLIT_AVAILABLE:
        # Ejecutar app streamlit normalmente (requiere comando `streamlit run`)
        run_streamlit_app(state)
    else:
        # Si no hay streamlit, comprobar si stdin es interactivo y elegir modo adecuado
        if STDIN_IS_INTERACTIVE:
            print("streamlit no está instalado — iniciando CLI interactivo.")
            print("Para usar la interfaz web, instala streamlit: pip install streamlit")
            run_interactive_cli_app(state)
        else:
            # Entorno no interactivo: ejecutar demo automatizado en lugar de usar input()
            print("streamlit no está instalado y el entorno es no interactivo.")
            print("Ejecutando demo automatizado (sin input). Si quieres interacción, ejecuta en una terminal TTY o instala streamlit.")
            run_demo_noninteractive(state)

# ------------------ FIN DEL ARCHIVO ------------------


streamlit no está instalado y el entorno es no interactivo.
Ejecutando demo automatizado (sin input). Si quieres interacción, ejecuta en una terminal TTY o instala streamlit.
=== Demo no interactiva del Gestor Inteligente de Proyecto ===
Fases detectadas:
 - Visión
 - Planeación
 - Ejecución
Siguiente tarea en Visión: (0) Definir objetivos del proyecto
Marcada como completada: Tarea 1 de Visión
Archivo de ejemplo asociado a la tarea 1 de Visión
Progreso en Visión: 1/3 (33%)
Estado guardado en: /tmp/project_state_demo.json

Log de acciones:
 - Siguiente tarea en 'Visión' -> Definir objetivos del proyecto
 - Tarea 0 en 'Visión' marcada como COMPLETADA
 - Archivo 'ejemplo.txt' subido para tarea 0 en 'Visión'
 - Estado guardado en /tmp/project_state_demo.json

Demo finalizada.
